In [1]:
# === 1. Import Required Libraries ===
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)
from imblearn.over_sampling import SMOTE

# Import Keras components from TensorFlow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Suppress unnecessary warnings
warnings.filterwarnings('ignore', category=UserWarning)
tf.get_logger().setLevel('ERROR') # Suppress TensorFlow informational messages

# === 2. Load Dataset ===
try:
    combined_dataset = pd.read_csv("/Users/administrator/Desktop/All desktop-data/Prof-Irshad Papers/2025-Papers/10K-Paper/Crop-IoT-Paper/IoT-Journal-Submission/Manuscript-R2/Code-For-Github/Combined_Data.csv")
except FileNotFoundError:
    print("Error: CSV file not found. Using a dummy dataframe for demonstration.")
    data = np.random.rand(5000, 9)
    columns = ['N', 'P', 'K', 'Temp', 'H', 'pH', 'R', 'WL', 'CH']
    combined_dataset = pd.DataFrame(data, columns=columns)
    combined_dataset['CH'] = np.random.randint(0, 2, 5000)

# === 3. Prepare, Split, Scale, and Balance Data ===
features = ['N', 'P', 'K', 'Temp', 'H', 'pH', 'R', 'WL']
n_features = len(features)
X = combined_dataset[features].values
y = combined_dataset['CH'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_scaled, y_train)

# === 4. Reshape Data for 1D CNN ===
# CNNs require an extra dimension for the "channels". For tabular data, this is 1.
# Shape changes from (num_samples, num_features) to (num_samples, num_features, 1)
X_train_reshaped = X_train_resampled.reshape(X_train_resampled.shape[0], n_features, 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], n_features, 1)


# === 5. Define a function to build the CNN model ===
def build_cnn_model():
    model = Sequential([
        # Input Layer: 1D Convolutional layer
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_features, 1)),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        
        # Flatten the output to feed into a Dense layer
        Flatten(),
        
        # Hidden Dense layer
        Dense(64, activation='relu'),
        
        # Output Layer: for binary classification
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# === 6. Run Simulation for CNN Model ===
EPOCHS = 163
acc_list, prec_list, rec_list, f1_list, auc_list = [], [], [], [], []

n_samples = len(X_train_reshaped)
print(f"\nStarting 1D CNN simulation for {EPOCHS} epochs...")
print("-" * 60)

for epoch in range(1, EPOCHS + 1):
    subset_size = int(n_samples * (epoch / EPOCHS))
    subset_size = max(10, subset_size) # Ensure a minimum number of samples
    
    X_subset = X_train_reshaped[:subset_size]
    y_subset = y_train_resampled[:subset_size]

    if len(np.unique(y_subset)) < 2:
        print(f"Epoch {epoch:03d}/{EPOCHS}: Skipped (not enough class diversity)")
        acc_list.append(acc_list[-1] if acc_list else 0.5)
        prec_list.append(prec_list[-1] if prec_list else 0)
        rec_list.append(rec_list[-1] if rec_list else 0)
        f1_list.append(f1_list[-1] if f1_list else 0)
        auc_list.append(auc_list[-1] if auc_list else 0.5)
        continue

    # --- Build and Train a new CNN Model on the current subset ---
    cnn_model = build_cnn_model()
    # Train the model for a few internal epochs. verbose=0 keeps the log clean.
    cnn_model.fit(X_subset, y_subset, epochs=10, batch_size=32, verbose=0)
    
    # --- Evaluate on the FULL, UNSEEN test set ---
    cnn_probs = cnn_model.predict(X_test_reshaped, verbose=0).flatten()
    y_pred = (cnn_probs >= 0.5).astype(int)
    
    # --- Calculate and Store Metrics ---
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = roc_auc_score(y_test, cnn_probs)

    acc_list.append(acc)
    prec_list.append(prec)
    rec_list.append(rec)
    f1_list.append(f1)
    auc_list.append(auc)
    
    print(f"Epoch {epoch:03d}/{EPOCHS} -> "
          f"Accuracy: {acc*100:.2f}%, Precision: {prec*100:.2f}%, "
          f"Recall: {rec*100:.2f}%, F1: {f1*100:.2f}%, AUC: {auc*100:.2f}%")


# === 7. Store History in a DataFrame ===
metrics_df = pd.DataFrame({
    'Epoch': range(1, EPOCHS + 1),
    'Accuracy': acc_list,
    'Precision': prec_list,
    'Recall': rec_list,
    'F1Score': f1_list,
    'AUC': auc_list
})

print("\n" + "="*40)
print("Simulation Complete.")
print(f"Final performance at Epoch {EPOCHS}:")
print(metrics_df.iloc[-1].to_string())
print("="*40)

2025-06-19 13:55:00.080617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



Starting 1D CNN simulation for 163 epochs...
------------------------------------------------------------
Epoch 001/163: Skipped (not enough class diversity)
Epoch 002/163: Skipped (not enough class diversity)
Epoch 003/163: Skipped (not enough class diversity)
Epoch 004/163: Skipped (not enough class diversity)
Epoch 005/163: Skipped (not enough class diversity)
Epoch 006/163: Skipped (not enough class diversity)
Epoch 007/163: Skipped (not enough class diversity)
Epoch 008/163: Skipped (not enough class diversity)
Epoch 009/163: Skipped (not enough class diversity)
Epoch 010/163: Skipped (not enough class diversity)
Epoch 011/163: Skipped (not enough class diversity)
Epoch 012/163: Skipped (not enough class diversity)
Epoch 013/163: Skipped (not enough class diversity)
Epoch 014/163: Skipped (not enough class diversity)
Epoch 015/163 -> Accuracy: 99.55%, Precision: 0.00%, Recall: 0.00%, F1: 0.00%, AUC: 83.79%
Epoch 016/163 -> Accuracy: 99.55%, Precision: 0.00%, Recall: 0.00%, F1: 0.

Epoch 096/163 -> Accuracy: 96.59%, Precision: 6.67%, Recall: 50.00%, F1: 11.76%, AUC: 96.92%
Epoch 097/163 -> Accuracy: 97.05%, Precision: 7.69%, Recall: 50.00%, F1: 13.33%, AUC: 97.49%
Epoch 098/163 -> Accuracy: 96.59%, Precision: 6.67%, Recall: 50.00%, F1: 11.76%, AUC: 97.72%
Epoch 099/163 -> Accuracy: 97.05%, Precision: 7.69%, Recall: 50.00%, F1: 13.33%, AUC: 97.15%
Epoch 100/163 -> Accuracy: 96.82%, Precision: 7.14%, Recall: 50.00%, F1: 12.50%, AUC: 97.15%
Epoch 101/163 -> Accuracy: 97.05%, Precision: 7.69%, Recall: 50.00%, F1: 13.33%, AUC: 97.60%
Epoch 102/163 -> Accuracy: 97.50%, Precision: 9.09%, Recall: 50.00%, F1: 15.38%, AUC: 97.95%
Epoch 103/163 -> Accuracy: 96.36%, Precision: 6.25%, Recall: 50.00%, F1: 11.11%, AUC: 97.26%
Epoch 104/163 -> Accuracy: 96.82%, Precision: 7.14%, Recall: 50.00%, F1: 12.50%, AUC: 97.60%
Epoch 105/163 -> Accuracy: 96.82%, Precision: 7.14%, Recall: 50.00%, F1: 12.50%, AUC: 98.17%
Epoch 106/163 -> Accuracy: 97.05%, Precision: 7.69%, Recall: 50.00%, F